In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import re
from collections import defaultdict
import plotly.io as pio

players = pd.read_csv('data/Player.csv')
player_atts = pd.read_csv('data/Player_Attributes.csv')
teams = pd.read_csv('data/Team.csv')
matches = pd.read_csv('data/Match.csv')
leagues = pd.read_csv('data/League.csv')
positions = pd.read_csv('data/PositionReference.csv')

player_atts['potential_rating_ratio'] = ((player_atts['potential'] / player_atts['overall_rating']) * 100)

In [ ]:
teams[teams['team_long_name']=='RCD Espanyol']

In [10]:
matches['date'] = pd.to_datetime(matches['date'])
rcde_matches = matches[matches['home_team_api_id']== 8558]

In [12]:
filtered_players_rcde = [col for col in rcde_matches.columns if re.match(r'home_player_\d+$', col)]

In [3]:
# Add column player name
player_atts = player_atts.merge(players[['player_api_id', 'player_name']], on='player_api_id', how='left')

In [4]:
# From the scatterplot
with open("data/promising_names.txt", "r") as f:
    promising_names = [line.strip() for line in f]
# Retain the most promising players
promising_players = player_atts[player_atts['player_name'].isin(promising_names)]

In [5]:
player_cols = [col for col in matches.columns if re.fullmatch(r'home_player_\d+', col) or re.fullmatch(r'away_player_\d+', col)]

# Step 2: For each player column, find its corresponding position column and merge
for col in player_cols:
    # Find the corresponding Y column (e.g., 'home_player_Y1' for 'home_player_1')
    y_col = col.replace('home_player_', 'home_player_Y')
    y_col_away = col.replace('away_player_', 'away_player_Y')

    matches[col] = matches[col].astype('Int64')

    # Merge with players table on player ID
    matches = matches.merge(
        players,
        left_on=col,
        right_on='player_api_id',
        how='left',
        suffixes=('', f'_{col}_info')
    )

    # Optional: Add position info from the Y column to the merged data
    matches[f'{col}_position'] = matches[y_col]

In [6]:
matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,weight_away_player_10_info,away_player_10_position,id_away_player_11_info,player_api_id_away_player_11_info,player_name_away_player_11_info,player_fifa_api_id_away_player_11_info,birthday_away_player_11_info,height_away_player_11_info,weight_away_player_11_info,away_player_11_position
0,1,1,1,2008/2009,1,17/08/2008 00:00,492473,9987,9993,1,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,2,1,1,2008/2009,1,16/08/2008 00:00,492474,10000,9994,0,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,3,1,1,2008/2009,1,16/08/2008 00:00,492475,9984,8635,0,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,4,1,1,2008/2009,1,17/08/2008 00:00,492476,9991,9998,5,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,5,1,1,2008/2009,1,16/08/2008 00:00,492477,7947,9985,1,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,22/09/2015 00:00,1992091,10190,10191,1,...,183.0,232110,9186.0,289732.0,Roman Buess,206431.0,1992-09-21 00:00:00,182.88,168.0,289732
25975,25976,24558,24558,2015/2016,9,23/09/2015 00:00,1992092,9824,10199,1,...,181.0,34082,4550.0,95257.0,Jakob Jantscher,187498.0,1989-01-08 00:00:00,177.80,154.0,95257
25976,25977,24558,24558,2015/2016,9,23/09/2015 00:00,1992093,9956,10179,2,...,154.0,451335,7725.0,275122.0,Moussa Konate,211321.0,1993-04-03 00:00:00,182.88,172.0,275122
25977,25978,24558,24558,2015/2016,9,22/09/2015 00:00,1992094,7896,10243,0,...,161.0,92252,3509.0,194532.0,Franck Etoundi,198591.0,1990-08-30 00:00:00,185.42,181.0,194532


In [ ]:
# can be deleted
rcde_player_atts = player_atts[player_atts['player_api_id'].isin(rcde_players['player_api_id'])]
# add name into the player attributes df
rcde_player_atts = rcde_player_atts.merge(rcde_players[['player_api_id', 'player_name']], on='player_api_id', how='left')

In [ ]:
# check whether it works
relevant_columns = [col for col in matches.columns if (
    ("home_player" in col or "away_player" in col) and
    ("_position" in col or "player_api_id" in col or "player_name" in col)
)]

# Subset
players_filtered = matches[relevant_columns]

import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Home and Away Player Info", dataframe=players_filtered)


In [8]:
# Extract relevant name and position columns again
filtered_players_df = pd.read_csv('data/filtered_player_info.csv')
name_cols = [col for col in filtered_players_df.columns if 'player_name_home_player_' in col]
pos_cols = [col for col in filtered_players_df.columns if re.fullmatch(r'home_player_\d+_position', col)]

player_position_dict = {}

for name_col in name_cols:
    player_number = re.search(r'\d+', name_col).group()
    pos_col = f'home_player_{player_number}_position'

    for name, pos in zip(filtered_players_df[name_col], filtered_players_df[pos_col]):
        if pd.notna(name) and pd.notna(pos):
            player_position_dict[name] = int(pos)


In [9]:
player_position_dict

{'Kenny van Hoevelen': 3,
 'Mohamed Sarr': 7,
 'Olivier Doll': 3,
 'Koen Daerden': 11,
 'Pieterjan Monteyne': 7,
 'Eric Deflandre': 3,
 "Karel D'Haene": 7,
 'Denis Viane': 7,
 'Frederic Jay': 3,
 'Olivier Deschacht': 3,
 'Daniel Calvo': 7,
 'Eric Matoukou': 3,
 'Nico van Kerckhoven': 3,
 'Damir Mirvic': 3,
 'Dario Smoje': 3,
 'Oguchi Onyewu': 3,
 'Didier Dheedene': 3,
 'Bernt Evens': 3,
 'Mvuezolo Muscal Musumbu': 7,
 'Joao Carlos': 10,
 'Jelle van Damme': 8,
 'Gonzague Vandooren': 7,
 'Michael Klukowski': 3,
 'Loris Reina': 3,
 'Michael Wiggers': 3,
 'Gerald Forschelet': 7,
 'Wouter Vrancken': 10,
 'Azubuike Oliseh': 3,
 'Brecht Verbrugghe': 7,
 'Quinton Fortune': 7,
 'Kristof Imschoot': 7,
 'Hans Cornelis': 3,
 'Steve Colpaert': 3,
 'Glenn Verbauwhede': 3,
 'Erlend Hanstveit': 3,
 'Stef Wils': 7,
 'Jonas Ivens': 7,
 'Jason Vandelannoite': 3,
 'Carl Hoefkens': 3,
 'Vincent Euvrard': 3,
 'Philippe Clement': 3,
 'Avi Strool': 3,
 'Jimmy Hempte': 7,
 'Ricardo Rocha': 3,
 'Mario Cantalupp

In [10]:
# only get the role_y information
pos_role_dict = {}
for position, role in zip(positions['player_pos_y'], positions['role_y']):
    pos_role_dict[position] = role

In [11]:
# map player position numbers and map to role
player_role_dict = {player: pos_role_dict.get(int(position), 'Unknown') for player, position in player_position_dict.items()}

In [12]:
# sort player names per rol on the soccer pitch
roles = defaultdict(list)
for player, role in player_role_dict.items():
    roles[role].append(player)

unique_roles = sorted(roles.keys())
role_counts = {role: len(players) for role, players in roles.items()}

gk_player_names = roles['GK']
bk_player_names = roles['BK']
mf_player_names = roles['MF']
fw_player_names = roles['FW']

In [14]:
# initalize dataframes to base the heatmaps on
bk_players = promising_players[promising_players['player_name'].isin(bk_player_names)]
mf_players = promising_players[promising_players['player_name'].isin(mf_player_names)]
fw_players = promising_players[promising_players['player_name'].isin(fw_player_names)]

In [16]:
fw_players.to_csv("fw_players.csv", index=False)

In [17]:
mf_players.to_csv("mf_players.csv", index=False)

In [18]:
bk_players.to_csv("bk_players.csv", index=False)

In [92]:
# reorder columns in all dfs
cols = promising_players.columns.tolist()
cols_to_move = ['player_name', 'potential_rating_ratio']
target_col = 'player_api_id'

for i in cols_to_move:
    cols.remove(i)
cols.insert(cols.index(target_col) + 1, cols_to_move[0])
cols.insert(cols.index(target_col) + 2, cols_to_move[1])

promising_player_atts = promising_players[cols]

In [98]:
pio.renderers.default = "notebook_connected"

In [116]:
def create_heatmap(df):
    disposable_cols = ['player_api_id', 'player_fifa_api_id', 'date', 'id',
                       'gk_diving', 'gk_kicking', 'gk_handling', 'gk_positioning', 'gk_reflexes']
    attribute_cols = df.select_dtypes(include='number').columns.difference(disposable_cols + ['potential_rating_ratio']).tolist()
    attribute_cols = ['potential_rating_ratio'] + attribute_cols

    pivot_df = df.pivot_table(index=None, columns='player_name', values=attribute_cols, aggfunc='mean')
    pivot_df = pivot_df.T
    pio.renderers.default = "notebook_connected"

    fig = go.Figure(
        data=go.Heatmap(
            z=pivot_df.values,
            x=pivot_df.columns,
            y=pivot_df.index,
            colorscale='Greens',
            colorbar=dict(title='Attribute Value'),
            hovertemplate='Player: %{x}<br>Attribute: %{y}<br>Value: %{z}<extra></extra>'
        )
    )

    fig.update_layout(
        xaxis=dict(title='Attributes', tickangle=45),
        yaxis=dict(title='Player Names'),
        autosize=True,
        height=600,
        width=1200
    )

    return fig


In [124]:
fw_heatmap = create_heatmap(fw_players)
bk_heatmap = create_heatmap(bk_players)
mf_heatmap = create_heatmap(mf_players)
